# **Example for Paper**: [Non-Euclidean Universal Approximation](https://arxiv.org/abs/2006.02341)

## Preping

We compare three models in this implementation.  Each are feed-forward networks of the same dimensions:
- **Good model**: repsects our assumptions
- **Bad model**: does not
- **Vanilla model**: is a naive feed-forward benchmark
#### Import Libraries

In [1]:
# Alert(s)
import smtplib

# CV
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelBinarizer

# DL: Tensorflow
import tensorflow as tf
from keras.utils.layer_utils import count_params
from tensorflow.python.framework import ops # Custome Tensorflow Functions
from tensorflow.keras.models import Sequential, Model
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.layers import Dense, Input
# DL: Tensorflow - Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K

# Evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Formatting:
import pandas as pd
import numpy as np

# Pre-Processing
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from scipy.special import expit

# Random Forest & Gradient Boosting (Arch. Construction)
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

# Structuring
from pathlib import Path

# Visulatization
import matplotlib.pyplot as plt

# Writing, Reading, Exporting, and Importing
#from sklearn.externals import joblib
import pickle

# Timing
import time

# Misc
import gc
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
import os

# Set-Seed
np.random.seed(2020)

Using TensorFlow backend.


#### Load Externally-Defined Functions

In [2]:
# Main Training Utility
exec(open('TCP_Util.py').read())
# Helper Functions Utility
exec(open('Optimal_Deep_Feature_and_Readout_Util.py').read())
# Extra Utilities
exec(open('Grid_Enhanced_Network.py').read())

#### Load Data

In [3]:
# load dataset
data_path = "./data/housing_complete.csv"
X = pd.read_csv(data_path)

# Parse/Prepare Data
X_train, y_train, X_test, y_test= prepare_data(data_path, True)

#### Check and Make Paths

In [4]:
Path('./outputs/models/').mkdir(parents=True, exist_ok=True)
Path('./outputs/models/Vanilla/').mkdir(parents=True, exist_ok=True)
Path('./outputs/models/Deep_Features/').mkdir(parents=True, exist_ok=True)
Path('./outputs/tables/').mkdir(parents=True, exist_ok=True)
Path('./outputs/results/').mkdir(parents=True, exist_ok=True)

---
---
---

# Good Model:
Build and train the good model:
$$
\rho \circ f\circ \phi:\mathbb{R}^m\rightarrow \mathbb{R}^n.
$$
 - $f$ is a shallow feed-forward network with ReLU activation.  
 - Readout: $\rho(x) = \operatorname{Leaky-ReLU}\bullet (\exp(\tilde{A}_n)x+\tilde{b}_n)\circ \dots \circ \operatorname{Leaky-ReLU}\bullet (\exp(\tilde{A}_1)x+\tilde{b}_1)$
 - Feature Map: $\phi(x) = \operatorname{Leaky-ReLU}\bullet (\exp(A_n)x+b_n)\circ \dots \circ\operatorname{Leaky-ReLU}\bullet (\exp(A_1)x+b_1)$

where $A_i,\tilde{A}_j$ are square matrices.  


The matrices $\exp(A_i)$, and $\exp(\tilde{A}_i)$ are therefore invertible since $\exp$ maps any square matrix into the associated [General Linear Group](https://en.wikipedia.org/wiki/General_linear_group).  

In [6]:
#------------------------------------------------------------------------------------------------#
#                                      Define Predictive Model                                   #
#------------------------------------------------------------------------------------------------#

def def_trainable_layers_Nice_Input_Output(height, Depth_Feature_Map, Depth_Readout_Map, learning_rate, input_dim, output_dim):
    #----------------------------#
    # Maximally Interacting Layer #
    #-----------------------------#
    # Initialize Inputs
    input_layer = tf.keras.Input(shape=(input_dim,))
    
    
    #------------------#
    # Deep Feature Map #
    #------------------#
    for i_feature_depth in range(Depth_Feature_Map):
        # First Layer
        if i_feature_depth == 0:
            deep_feature_map = fullyConnected_Dense_Invertible(input_dim)(input_layer)
            deep_feature_map = tf.nn.leaky_relu(deep_feature_map)
        else:
            deep_feature_map = fullyConnected_Dense_Invertible(input_dim)(deep_feature_map)
            deep_feature_map = tf.nn.leaky_relu(deep_feature_map)
    
    #------------------#
    #   Core Layers    #
    #------------------#
    core_layers = fullyConnected_Dense(height)(deep_feature_map)
    # Activation
    core_layers = tf.nn.relu(core_layers)
    # Affine Layer (Dense Fully Connected)
    output_layers = fullyConnected_Dense(output_dim)(core_layers)
    
    
    #------------------#
    #  Readout Layers  #
    #------------------#   
    for i_depth_readout in range(Depth_Readout_Map):
        # First Layer
        if i_feature_depth == 0:
            output_layers = fullyConnected_Dense_Invertible(output_dim)(output_layers)
            output_layers = tf.nn.leaky_relu(output_layers)
        else:
            output_layers = fullyConnected_Dense_Invertible(output_dim)(output_layers)
            output_layers = tf.nn.leaky_relu(output_layers)
    
    
    # Define Input/Output Relationship (Arch.)
    trainable_layers_model = tf.keras.Model(input_layer, output_layers)
    
    
    #----------------------------------#
    # Define Optimizer & Compile Archs.
    #----------------------------------#
    opt = Adam(lr=learning_rate)
    trainable_layers_model.compile(optimizer=opt, loss="mae", metrics=["mse", "mae", "mape"])

    return trainable_layers_model

#------------------------------------------------------------------------------------------------#
#                                      Build Predictive Model                                    #
#------------------------------------------------------------------------------------------------#

def build_and_predict_nice_model(n_folds , n_jobs):

    # Deep Feature Network
    Nice_Model_CV = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=def_trainable_layers_Nice_Input_Output, verbose=True)
    
    # Randomized CV
    Nice_Model_CVer = RandomizedSearchCV(estimator=Nice_Model_CV, 
                                    n_jobs=n_jobs,
                                    cv=KFold(CV_folds, random_state=2020, shuffle=True),
                                    param_distributions=param_grid_Nice_Nets,
                                    n_iter=n_iter,
                                    return_train_score=True,
                                    random_state=2020,
                                    verbose=10)
    
    # Fit
    Nice_Model_CVer.fit(X_train,y_train)

    # Write Predictions
    y_hat_train = Nice_Model_CVer.predict(X_train)
    y_hat_test = Nice_Model_CVer.predict(X_test)
    
    # Return Values
    return y_hat_train, y_hat_test

# Update User
#-------------#
print('Built Model')

Built Model


### Make Predictions

In [7]:
# Initialize & User Updates
#--------------------------#
y_hat_train_good, y_hat_test_good = build_and_predict_nice_model(n_folds = 2, n_jobs = 2)
print('Cross-Validated: Good Model')


# Results with Nice Model
#------------------------#
Train_Good = y_hat_train_good - y_train
Test_Good = y_hat_test_good - y_test
score_Train_good = np.mean(np.abs(Train_Good))
score_Test_good = np.mean(np.abs(Test_Good))
score_Train_good_RMSE = np.mean((Train_Good)**2)
score_Test_good_RMSE = np.mean((Test_Good)**2)

# Performance Metrics
#----------------------#
performance_out = pd.DataFrame({
'Good': np.array([np.mean(score_Train_good),score_Train_good_RMSE,np.mean(score_Test_good),score_Test_good_RMSE])
},index=['MAE: Train','MAE: Test','RMSE: Train','RMSE: Test'])

# Write Results
#---------------#
# LaTeX
performance_out.to_latex('./outputs/results/Performance.txt')
# Write to Txt
cur_path = os.path.expanduser('./outputs/results/Performance_text.txt')
with open(cur_path, "w") as f:
    f.write(str(performance_out))

Fitting 2 folds for each of 1 candidates, totalling 2 fits


/scratch/users/kratsioa/.local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 1 is smaller than n_iter=5. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed: 31.2min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed: 31.2min finished


Train on 14447 samples
Epoch 1/1200
14447/14447 [==============================] - 11s 735us/sample - loss: 1.4200 - mse: 3.3168 - mae: 1.4200 - mape: 78.1724
Epoch 2/1200
14447/14447 [==============================] - 3s 196us/sample - loss: 0.8597 - mse: 1.2396 - mae: 0.8597 - mape: 55.1039
Epoch 3/1200
14447/14447 [==============================] - 3s 211us/sample - loss: 0.8066 - mse: 1.0966 - mae: 0.8066 - mape: 51.1332
Epoch 4/1200
14447/14447 [==============================] - 3s 197us/sample - loss: 0.7755 - mse: 1.0185 - mae: 0.7755 - mape: 49.3045
Epoch 5/1200
14447/14447 [==============================] - 3s 186us/sample - loss: 0.7591 - mse: 0.9867 - mae: 0.7591 - mape: 47.9090
Epoch 6/1200
14447/14447 [==============================] - 3s 206us/sample - loss: 0.7220 - mse: 0.8933 - mae: 0.7220 - mape: 45.1599
Epoch 7/1200
14447/14447 [==============================] - 3s 178us/sample - loss: 0.7167 - mse: 0.8912 - mae: 0.7167 - mape: 44.4887
Epoch 8/1200
14447/14447 [=====

14447/14447 [==============================] - 2s 113us/sample - loss: 0.4861 - mse: 0.4597 - mae: 0.4861 - mape: 27.9672
Epoch 62/1200
14447/14447 [==============================] - 2s 115us/sample - loss: 0.4929 - mse: 0.4698 - mae: 0.4929 - mape: 28.4559
Epoch 63/1200
14447/14447 [==============================] - 2s 118us/sample - loss: 0.4938 - mse: 0.4734 - mae: 0.4938 - mape: 28.6775
Epoch 64/1200
14447/14447 [==============================] - 2s 130us/sample - loss: 0.4850 - mse: 0.4581 - mae: 0.4850 - mape: 27.8707
Epoch 65/1200
14447/14447 [==============================] - 2s 120us/sample - loss: 0.4875 - mse: 0.4643 - mae: 0.4875 - mape: 28.1545
Epoch 66/1200
14447/14447 [==============================] - 2s 131us/sample - loss: 0.4888 - mse: 0.4626 - mae: 0.4888 - mape: 28.3043
Epoch 67/1200
14447/14447 [==============================] - 2s 149us/sample - loss: 0.4857 - mse: 0.4603 - mae: 0.4857 - mape: 27.9086
Epoch 68/1200
14447/14447 [==============================] - 2

14447/14447 [==============================] - 2s 145us/sample - loss: 0.4590 - mse: 0.4298 - mae: 0.4590 - mape: 25.9995
Epoch 122/1200
14447/14447 [==============================] - 2s 122us/sample - loss: 0.4565 - mse: 0.4253 - mae: 0.4565 - mape: 25.9432
Epoch 123/1200
14447/14447 [==============================] - 2s 112us/sample - loss: 0.4585 - mse: 0.4309 - mae: 0.4585 - mape: 26.0514
Epoch 124/1200
14447/14447 [==============================] - 2s 111us/sample - loss: 0.4627 - mse: 0.4321 - mae: 0.4627 - mape: 26.3408
Epoch 125/1200
14447/14447 [==============================] - 2s 113us/sample - loss: 0.4652 - mse: 0.4390 - mae: 0.4652 - mape: 26.3419
Epoch 126/1200
14447/14447 [==============================] - 2s 112us/sample - loss: 0.4521 - mse: 0.4190 - mae: 0.4521 - mape: 25.6457
Epoch 127/1200
14447/14447 [==============================] - 2s 135us/sample - loss: 0.4636 - mse: 0.4333 - mae: 0.4636 - mape: 26.3560
Epoch 128/1200
14447/14447 [============================

...

14447/14447 [==============================] - 2s 121us/sample - loss: 0.3924 - mse: 0.3385 - mae: 0.3924 - mape: 21.6591
Epoch 948/1200
14447/14447 [==============================] - 2s 119us/sample - loss: 0.3884 - mse: 0.3329 - mae: 0.3884 - mape: 21.5276
Epoch 949/1200
14447/14447 [==============================] - 2s 109us/sample - loss: 0.3912 - mse: 0.3362 - mae: 0.3912 - mape: 21.6894
Epoch 950/1200
14447/14447 [==============================] - 2s 108us/sample - loss: 0.3901 - mse: 0.3356 - mae: 0.3901 - mape: 21.6467
Epoch 951/1200
14447/14447 [==============================] - 2s 108us/sample - loss: 0.3901 - mse: 0.3371 - mae: 0.3901 - mape: 21.5703
Epoch 952/1200
14447/14447 [==============================] - 2s 106us/sample - loss: 0.3907 - mse: 0.3354 - mae: 0.3907 - mape: 21.6904
Epoch 953/1200
14447/14447 [==============================] - 2s 108us/sample - loss: 0.3897 - mse: 0.3360 - mae: 0.3897 - mape: 21.4829
Epoch 954/1200
14447/14447 [============================

14447/14447 [==============================] - 2s 108us/sample - loss: 0.3901 - mse: 0.3357 - mae: 0.3901 - mape: 21.5966
Epoch 1007/1200
14447/14447 [==============================] - 2s 107us/sample - loss: 0.3900 - mse: 0.3363 - mae: 0.3900 - mape: 21.6342
Epoch 1008/1200
14447/14447 [==============================] - 2s 108us/sample - loss: 0.3880 - mse: 0.3347 - mae: 0.3880 - mape: 21.4899
Epoch 1009/1200
14447/14447 [==============================] - 2s 107us/sample - loss: 0.3891 - mse: 0.3354 - mae: 0.3891 - mape: 21.5147
Epoch 1010/1200
14447/14447 [==============================] - 2s 108us/sample - loss: 0.3889 - mse: 0.3346 - mae: 0.3889 - mape: 21.5138
Epoch 1011/1200
14447/14447 [==============================] - 2s 109us/sample - loss: 0.3885 - mse: 0.3350 - mae: 0.3885 - mape: 21.4895
Epoch 1012/1200
14447/14447 [==============================] - 2s 108us/sample - loss: 0.3880 - mse: 0.3344 - mae: 0.3880 - mape: 21.5299
Epoch 1013/1200
14447/14447 [=====================

14447/14447 [==============================] - 2s 107us/sample - loss: 0.3888 - mse: 0.3348 - mae: 0.3888 - mape: 21.5255
Epoch 1066/1200
14447/14447 [==============================] - 2s 108us/sample - loss: 0.3886 - mse: 0.3324 - mae: 0.3886 - mape: 21.4467
Epoch 1067/1200
14447/14447 [==============================] - 2s 107us/sample - loss: 0.3841 - mse: 0.3291 - mae: 0.3841 - mape: 21.2798
Epoch 1068/1200
14447/14447 [==============================] - 2s 108us/sample - loss: 0.3854 - mse: 0.3317 - mae: 0.3854 - mape: 21.2753
Epoch 1069/1200
14447/14447 [==============================] - 2s 106us/sample - loss: 0.3876 - mse: 0.3316 - mae: 0.3876 - mape: 21.4260
Epoch 1070/1200
14447/14447 [==============================] - 2s 106us/sample - loss: 0.3877 - mse: 0.3342 - mae: 0.3877 - mape: 21.4583
Epoch 1071/1200
14447/14447 [==============================] - 2s 107us/sample - loss: 0.3848 - mse: 0.3292 - mae: 0.3848 - mape: 21.1926
Epoch 1072/1200
14447/14447 [=====================

14447/14447 [==============================] - 2s 110us/sample - loss: 0.3866 - mse: 0.3314 - mae: 0.3866 - mape: 21.4119
Epoch 1125/1200
14447/14447 [==============================] - 2s 108us/sample - loss: 0.3857 - mse: 0.3295 - mae: 0.3857 - mape: 21.3803
Epoch 1126/1200
14447/14447 [==============================] - 2s 107us/sample - loss: 0.3861 - mse: 0.3307 - mae: 0.3861 - mape: 21.3833
Epoch 1127/1200
14447/14447 [==============================] - 2s 110us/sample - loss: 0.3870 - mse: 0.3330 - mae: 0.3870 - mape: 21.3593
Epoch 1128/1200
14447/14447 [==============================] - 2s 108us/sample - loss: 0.3846 - mse: 0.3306 - mae: 0.3846 - mape: 21.3012
Epoch 1129/1200
14447/14447 [==============================] - 2s 108us/sample - loss: 0.3822 - mse: 0.3272 - mae: 0.3822 - mape: 21.0676
Epoch 1130/1200
14447/14447 [==============================] - 2s 109us/sample - loss: 0.3850 - mse: 0.3304 - mae: 0.3850 - mape: 21.2969
Epoch 1131/1200
14447/14447 [=====================

14447/14447 [==============================] - 2s 109us/sample - loss: 0.3839 - mse: 0.3299 - mae: 0.3839 - mape: 21.2298
Epoch 1184/1200
14447/14447 [==============================] - 2s 114us/sample - loss: 0.3849 - mse: 0.3289 - mae: 0.3849 - mape: 21.3066
Epoch 1185/1200
14447/14447 [==============================] - 2s 109us/sample - loss: 0.3860 - mse: 0.3299 - mae: 0.3860 - mape: 21.3891
Epoch 1186/1200
14447/14447 [==============================] - 2s 131us/sample - loss: 0.3842 - mse: 0.3284 - mae: 0.3842 - mape: 21.1579
Epoch 1187/1200
14447/14447 [==============================] - 2s 121us/sample - loss: 0.3835 - mse: 0.3295 - mae: 0.3835 - mape: 21.1926
Epoch 1188/1200
14447/14447 [==============================] - 2s 117us/sample - loss: 0.3836 - mse: 0.3268 - mae: 0.3836 - mape: 21.2462
Epoch 1189/1200
14447/14447 [==============================] - 2s 111us/sample - loss: 0.3836 - mse: 0.3274 - mae: 0.3836 - mape: 21.2360
Epoch 1190/1200
14447/14447 [=====================

# Bad Model:
Build and train the *bad* model:
$$
\rho \circ f\circ \phi:\mathbb{R}^m\rightarrow \mathbb{R}^n.
$$
 - $f$ is a shallow feed-forward network with ReLU activation.  
 - Readout: $\rho(x) = \operatorname{ReLU}\bullet (\tilde{A}_nx+\tilde{b}_n)\circ \dots \circ \operatorname{ReLU}\bullet (\tilde{A}_1x+\tilde{b}_1)$
 - Feature Map: $\phi(x) = \operatorname{ReLU}\bullet (A_nx+b_n)\circ \dots \circ\operatorname{ReLU}\bullet (A_1x+b_1)$

where $A_i,\tilde{A}_j$ are square matrices.  The maps $\rho$ and $\phi$ are neither injective nor are they surjective since $x\mapsto \operatorname{ReLU}(x)$ is neither injective nor surjective as a map from $\mathbb{R}^k$ to $\mathbb{R}^k$; where $m=n$.  

*Note*:  The key point here is that the input and output maps are forced to be of the same dimension.  Note that, this also violates the minimal bounds derivated in [this paper](https://arxiv.org/abs/1710.11278) for deep ReLU networks.  

In [8]:
#------------------------------------------------------------------------------------------------#
#                                      Define Predictive Model                                   #
#------------------------------------------------------------------------------------------------#

def def_trainable_layers_Bad_Input_Output(height, Depth_Feature_Map, Depth_Readout_Map, learning_rate, input_dim, output_dim):
    #----------------------------#
    # Maximally Interacting Layer #
    #-----------------------------#
    # Initialize Inputs
    input_layer = tf.keras.Input(shape=(input_dim,))
    
    
    #------------------#
    # Deep Feature Map #
    #------------------#
    for i_feature_depth in range(Depth_Feature_Map):
        # First Layer
        if i_feature_depth == 0:
            deep_feature_map = fullyConnected_Dense(input_dim)(input_layer)
            deep_feature_map = tf.nn.relu(deep_feature_map)
        else:
            deep_feature_map = fullyConnected_Dense(input_dim)(deep_feature_map)
            deep_feature_map = tf.nn.relu(deep_feature_map)
    
    #------------------#
    #   Core Layers    #
    #------------------#
    core_layers = fullyConnected_Dense(height)(deep_feature_map)
    # Activation
    core_layers = tf.nn.relu(core_layers)
    # Affine Layer (Dense Fully Connected)
    output_layers = fullyConnected_Dense(output_dim)(core_layers)
    
    
    #------------------#
    #  Readout Layers  #
    #------------------#   
    for i_depth_readout in range(Depth_Readout_Map):
        # First Layer
        if i_feature_depth == 0:
            output_layers = fullyConnected_Dense(output_dim)(output_layers)
            output_layers = tf.nn.relu(output_layers)
        else:
            output_layers = fullyConnected_Dense(output_dim)(output_layers)
            output_layers = tf.nn.relu(output_layers)
    
    
    # Define Input/Output Relationship (Arch.)
    trainable_layers_model = tf.keras.Model(input_layer, output_layers)
    
    
    #----------------------------------#
    # Define Optimizer & Compile Archs.
    #----------------------------------#
    opt = Adam(lr=learning_rate)
    trainable_layers_model.compile(optimizer=opt, loss="mae", metrics=["mse", "mae", "mape"])

    return trainable_layers_model

#------------------------------------------------------------------------------------------------#
#                                      Build Predictive Model                                    #
#------------------------------------------------------------------------------------------------#

def build_and_predict_bad_model(n_folds , n_jobs):

    # Deep Feature Network
    Bad_Model_CV = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=def_trainable_layers_Bad_Input_Output, verbose=True)
    
    # Randomized CV
    Bad_Model_CVer = RandomizedSearchCV(estimator=Bad_Model_CV, 
                                    n_jobs=n_jobs,
                                    cv=KFold(CV_folds, random_state=2020, shuffle=True),
                                    param_distributions=param_grid_Nice_Nets,
                                    n_iter=n_iter,
                                    return_train_score=True,
                                    random_state=2020,
                                    verbose=10)
    
    # Fit
    Bad_Model_CVer.fit(X_train,y_train)

    # Write Predictions
    y_hat_train = Bad_Model_CVer.predict(X_train)
    y_hat_test = Bad_Model_CVer.predict(X_test)
    
    # Return Values
    return y_hat_train, y_hat_test

# Update User
#-------------#
print('Built Bad Model')

Built Bad Model


In [9]:
# Initialize & User Updates
#--------------------------#
y_hat_train_bad, y_hat_test_bad = build_and_predict_bad_model(n_folds = 2, n_jobs = 2)
print('Cross-Validated: Vanilla Model')

# Results with Bad Model
#-----------------------#
Train_Bad = y_hat_train_bad - y_train
Test_Bad = y_hat_test_bad - y_test
score_Train_bad = np.mean(np.abs(Train_Bad))
score_Test_bad = np.mean(np.abs(Test_Bad))
score_Train_bad_RMSE = np.mean((Train_Bad)**2)
score_Test_bad_RMSE = np.mean((Test_Bad)**2)

# Performance Metrics
#----------------------#
performance_out = pd.DataFrame({
'Good': np.array([np.mean(score_Train_good),score_Train_good_RMSE,np.mean(score_Test_good),score_Test_good_RMSE]),
'Bad': np.array([np.mean(score_Train_bad),score_Train_bad_RMSE,np.mean(score_Test_bad),score_Test_bad_RMSE])
},index=['MAE: Train','MAE: Test','RMSE: Train','RMSE: Test'])

# Write Results
#---------------#
# LaTeX
performance_out.to_latex('./outputs/results/Performance.txt')
# Write to Txt
cur_path = os.path.expanduser('./outputs/results/Performance_text.txt')
with open(cur_path, "w") as f:
    f.write(str(performance_out))

Fitting 2 folds for each of 1 candidates, totalling 2 fits


/scratch/users/kratsioa/.local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 1 is smaller than n_iter=5. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed: 12.5min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed: 12.5min finished


Train on 14447 samples
Epoch 1/1200
14447/14447 [==============================] - 1s 101us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 2/1200
14447/14447 [==============================] - 1s 62us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 3/1200
14447/14447 [==============================] - 1s 60us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 4/1200
14447/14447 [==============================] - 1s 61us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 5/1200
14447/14447 [==============================] - 1s 60us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 6/1200
14447/14447 [==============================] - 1s 65us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 7/1200
14447/14447 [==============================] - 1s 61us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 8/1200
14447/14447 [=====

14447/14447 [==============================] - 1s 60us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 62/1200
14447/14447 [==============================] - 1s 61us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 63/1200
14447/14447 [==============================] - 1s 60us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 64/1200
14447/14447 [==============================] - 1s 61us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 65/1200
14447/14447 [==============================] - 1s 60us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 66/1200
14447/14447 [==============================] - 1s 60us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 67/1200
14447/14447 [==============================] - 1s 61us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 68/1200
14447/14447 [==============================] - 1

14447/14447 [==============================] - 1s 69us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 122/1200
14447/14447 [==============================] - 1s 64us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 123/1200
14447/14447 [==============================] - 1s 63us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 124/1200
14447/14447 [==============================] - 1s 62us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 125/1200
14447/14447 [==============================] - 1s 61us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 126/1200
14447/14447 [==============================] - 1s 60us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 127/1200
14447/14447 [==============================] - 1s 60us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 128/1200
14447/14447 [============================

...

14447/14447 [==============================] - 1s 62us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 948/1200
14447/14447 [==============================] - 1s 62us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 949/1200
14447/14447 [==============================] - 1s 64us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 950/1200
14447/14447 [==============================] - 1s 66us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 951/1200
14447/14447 [==============================] - 1s 66us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 952/1200
14447/14447 [==============================] - 1s 66us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 953/1200
14447/14447 [==============================] - 1s 67us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 954/1200
14447/14447 [============================

14447/14447 [==============================] - 1s 66us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1007/1200
14447/14447 [==============================] - 1s 63us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1008/1200
14447/14447 [==============================] - 1s 63us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1009/1200
14447/14447 [==============================] - 1s 76us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1010/1200
14447/14447 [==============================] - 1s 69us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1011/1200
14447/14447 [==============================] - 1s 65us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1012/1200
14447/14447 [==============================] - 1s 65us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1013/1200
14447/14447 [=====================

14447/14447 [==============================] - 1s 62us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1066/1200
14447/14447 [==============================] - 1s 62us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1067/1200
14447/14447 [==============================] - 1s 61us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1068/1200
14447/14447 [==============================] - 1s 67us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1069/1200
14447/14447 [==============================] - 1s 65us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1070/1200
14447/14447 [==============================] - 1s 65us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1071/1200
14447/14447 [==============================] - 1s 69us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1072/1200
14447/14447 [=====================

14447/14447 [==============================] - 2s 128us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1125/1200
14447/14447 [==============================] - 2s 118us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1126/1200
14447/14447 [==============================] - 1s 99us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1127/1200
14447/14447 [==============================] - 1s 81us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1128/1200
14447/14447 [==============================] - 1s 69us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1129/1200
14447/14447 [==============================] - 1s 90us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1130/1200
14447/14447 [==============================] - 1s 70us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1131/1200
14447/14447 [===================

14447/14447 [==============================] - 1s 63us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1184/1200
14447/14447 [==============================] - 1s 61us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1185/1200
14447/14447 [==============================] - 1s 62us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1186/1200
14447/14447 [==============================] - 1s 70us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1187/1200
14447/14447 [==============================] - 1s 62us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1188/1200
14447/14447 [==============================] - 1s 62us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1189/1200
14447/14447 [==============================] - 1s 65us/sample - loss: 2.0736 - mse: 5.6372 - mae: 2.0736 - mape: 100.0000
Epoch 1190/1200
14447/14447 [=====================

# Vanilla Model

In [10]:
#------------------------------------------------------------------------------------------------#
#                                      Define Predictive Model                                   #
#------------------------------------------------------------------------------------------------#

def def_trainable_layers_Vanilla(height, Depth_Feature_Map, Depth_Readout_Map, learning_rate, input_dim, output_dim):
    #----------------------------#
    # Maximally Interacting Layer #
    #-----------------------------#
    # Initialize Inputs
    input_layer = tf.keras.Input(shape=(input_dim,))
    
    #------------------#
    #   Core Layers    #
    #------------------#
    core_layers = fullyConnected_Dense(height)(input_layer)
    # Activation
    core_layers = tf.nn.relu(core_layers)
    # Affine Layer (Dense Fully Connected)
    output_layers = fullyConnected_Dense(output_dim)(core_layers)
    
    
    # Define Input/Output Relationship (Arch.)
    trainable_layers_model = tf.keras.Model(input_layer, output_layers)
    
    
    #----------------------------------#
    # Define Optimizer & Compile Archs.
    #----------------------------------#
    opt = Adam(lr=learning_rate)
    trainable_layers_model.compile(optimizer=opt, loss="mae", metrics=["mse", "mae", "mape"])

    return trainable_layers_model

#------------------------------------------------------------------------------------------------#
#                                      Build Predictive Model                                    #
#------------------------------------------------------------------------------------------------#

def build_and_predict_Vanilla_model(n_folds , n_jobs):

    # Deep Feature Network
    Vanilla_Model_CV = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=def_trainable_layers_Vanilla, verbose=True)
    
    # Randomized CV
    Vanilla_Model_CVer = RandomizedSearchCV(estimator=Vanilla_Model_CV, 
                                    n_jobs=n_jobs,
                                    cv=KFold(CV_folds, random_state=2020, shuffle=True),
                                    param_distributions=param_grid_Nice_Nets,
                                    n_iter=n_iter,
                                    return_train_score=True,
                                    random_state=2020,
                                    verbose=10)
    
    # Fit
    Vanilla_Model_CVer.fit(X_train,y_train)

    # Write Predictions
    y_hat_train = Vanilla_Model_CVer.predict(X_train)
    y_hat_test = Vanilla_Model_CVer.predict(X_test)
    
    # Return Values
    return y_hat_train, y_hat_test

# Update User
#-------------#
print('Built Vanilla Model')

Built Vanilla Model


### Make Predictions

In [11]:
# Initialize & User Updates
#--------------------------#
y_hat_train_Vanilla, y_hat_test_Vanilla = build_and_predict_Vanilla_model(n_folds = 2, n_jobs = 2)
print('Cross-Validated: Vanilla Model')

Fitting 2 folds for each of 1 candidates, totalling 2 fits


/scratch/users/kratsioa/.local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 1 is smaller than n_iter=5. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  8.4min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  8.4min finished


Train on 14447 samples
Epoch 1/1200
14447/14447 [==============================] - 1s 61us/sample - loss: 2.8273 - mse: 68.6432 - mae: 2.8273 - mape: 165.3005
Epoch 2/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.8737 - mse: 1.4037 - mae: 0.8737 - mape: 55.6617
Epoch 3/1200
14447/14447 [==============================] - 1s 38us/sample - loss: 0.8355 - mse: 1.3110 - mae: 0.8355 - mape: 53.3794
Epoch 4/1200
14447/14447 [==============================] - 1s 38us/sample - loss: 0.7747 - mse: 1.1265 - mae: 0.7747 - mape: 48.9754
Epoch 5/1200
14447/14447 [==============================] - 1s 37us/sample - loss: 0.7237 - mse: 0.9645 - mae: 0.7237 - mape: 45.2587
Epoch 6/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.7060 - mse: 0.9711 - mae: 0.7060 - mape: 43.6286
Epoch 7/1200
14447/14447 [==============================] - 1s 38us/sample - loss: 0.6654 - mse: 0.8284 - mae: 0.6654 - mape: 40.3683
Epoch 8/1200
14447/14447 [===========

14447/14447 [==============================] - 1s 38us/sample - loss: 0.5931 - mse: 0.6602 - mae: 0.5931 - mape: 35.0313
Epoch 62/1200
14447/14447 [==============================] - 1s 38us/sample - loss: 0.5879 - mse: 0.6662 - mae: 0.5879 - mape: 34.8571
Epoch 63/1200
14447/14447 [==============================] - 1s 38us/sample - loss: 0.5748 - mse: 0.6275 - mae: 0.5748 - mape: 33.5934
Epoch 64/1200
14447/14447 [==============================] - 1s 38us/sample - loss: 0.5986 - mse: 0.6702 - mae: 0.5986 - mape: 35.2547
Epoch 65/1200
14447/14447 [==============================] - 1s 38us/sample - loss: 0.5879 - mse: 0.6710 - mae: 0.5879 - mape: 34.7480
Epoch 66/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.6107 - mse: 0.7177 - mae: 0.6107 - mape: 35.9885
Epoch 67/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.5690 - mse: 0.6110 - mae: 0.5690 - mape: 33.4080
Epoch 68/1200
14447/14447 [==============================] - 1s 47us/

14447/14447 [==============================] - 1s 39us/sample - loss: 0.5550 - mse: 0.5898 - mae: 0.5550 - mape: 32.6670
Epoch 122/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.5814 - mse: 0.6349 - mae: 0.5814 - mape: 34.3048
Epoch 123/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.5682 - mse: 0.6115 - mae: 0.5682 - mape: 33.3882
Epoch 124/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.5680 - mse: 0.6118 - mae: 0.5680 - mape: 33.4455
Epoch 125/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.5721 - mse: 0.6234 - mae: 0.5721 - mape: 33.6514
Epoch 126/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.5615 - mse: 0.5948 - mae: 0.5615 - mape: 32.9334
Epoch 127/1200
14447/14447 [==============================] - 1s 40us/sample - loss: 0.5588 - mse: 0.5835 - mae: 0.5588 - mape: 32.8470
Epoch 128/1200
14447/14447 [==============================] - 1

...

14447/14447 [==============================] - 1s 38us/sample - loss: 0.4803 - mse: 0.4596 - mae: 0.4803 - mape: 27.3458
Epoch 960/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.4636 - mse: 0.4278 - mae: 0.4636 - mape: 26.3674
Epoch 961/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.4676 - mse: 0.4364 - mae: 0.4676 - mape: 26.5186
Epoch 962/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.4682 - mse: 0.4394 - mae: 0.4682 - mape: 26.5193
Epoch 963/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.4911 - mse: 0.4777 - mae: 0.4911 - mape: 28.0948
Epoch 964/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.4719 - mse: 0.4382 - mae: 0.4719 - mape: 26.7612
Epoch 965/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.4740 - mse: 0.4406 - mae: 0.4740 - mape: 27.0451
Epoch 966/1200
14447/14447 [==============================] - 1

14447/14447 [==============================] - 1s 40us/sample - loss: 0.4706 - mse: 0.4429 - mae: 0.4706 - mape: 26.6593
Epoch 1020/1200
14447/14447 [==============================] - 1s 41us/sample - loss: 0.4699 - mse: 0.4388 - mae: 0.4699 - mape: 26.7656
Epoch 1021/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.4952 - mse: 0.4746 - mae: 0.4952 - mape: 28.3112
Epoch 1022/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.4745 - mse: 0.4509 - mae: 0.4745 - mape: 26.9868
Epoch 1023/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.4722 - mse: 0.4434 - mae: 0.4722 - mape: 26.9081
Epoch 1024/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.4675 - mse: 0.4315 - mae: 0.4675 - mape: 26.4766
Epoch 1025/1200
14447/14447 [==============================] - 1s 40us/sample - loss: 0.4654 - mse: 0.4358 - mae: 0.4654 - mape: 26.2846
Epoch 1026/1200
14447/14447 [============================

14447/14447 [==============================] - 1s 38us/sample - loss: 0.4791 - mse: 0.4545 - mae: 0.4791 - mape: 27.2496
Epoch 1079/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.4725 - mse: 0.4470 - mae: 0.4725 - mape: 26.7182
Epoch 1080/1200
14447/14447 [==============================] - 1s 38us/sample - loss: 0.4666 - mse: 0.4318 - mae: 0.4666 - mape: 26.4147
Epoch 1081/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.4677 - mse: 0.4397 - mae: 0.4677 - mape: 26.4976
Epoch 1082/1200
14447/14447 [==============================] - 1s 38us/sample - loss: 0.4575 - mse: 0.4220 - mae: 0.4575 - mape: 25.7759
Epoch 1083/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.4666 - mse: 0.4334 - mae: 0.4666 - mape: 26.5394
Epoch 1084/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.4768 - mse: 0.4498 - mae: 0.4768 - mape: 27.0347
Epoch 1085/1200
14447/14447 [============================

14447/14447 [==============================] - 1s 39us/sample - loss: 0.4684 - mse: 0.4320 - mae: 0.4684 - mape: 26.5066
Epoch 1138/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.4696 - mse: 0.4349 - mae: 0.4696 - mape: 26.6334
Epoch 1139/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.4638 - mse: 0.4296 - mae: 0.4638 - mape: 26.2104
Epoch 1140/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.4572 - mse: 0.4174 - mae: 0.4572 - mape: 25.7407
Epoch 1141/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.4657 - mse: 0.4356 - mae: 0.4657 - mape: 26.3674
Epoch 1142/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.4589 - mse: 0.4240 - mae: 0.4589 - mape: 26.0073
Epoch 1143/1200
14447/14447 [==============================] - 1s 40us/sample - loss: 0.4543 - mse: 0.4195 - mae: 0.4543 - mape: 25.4398
Epoch 1144/1200
14447/14447 [============================

14447/14447 [==============================] - 1s 40us/sample - loss: 0.4684 - mse: 0.4356 - mae: 0.4684 - mape: 26.3857
Epoch 1197/1200
14447/14447 [==============================] - 1s 40us/sample - loss: 0.4767 - mse: 0.4649 - mae: 0.4767 - mape: 27.0052
Epoch 1198/1200
14447/14447 [==============================] - 1s 39us/sample - loss: 0.4610 - mse: 0.4282 - mae: 0.4610 - mape: 25.9895
Epoch 1199/1200
14447/14447 [==============================] - 1s 38us/sample - loss: 0.4524 - mse: 0.4119 - mae: 0.4524 - mape: 25.4571
Epoch 1200/1200
6193/6193 [==============================] - 0s 20us/sample
Cross-Validated: Vanilla Model


# Record Predictions/ Comparisons
Generate Classes

In [12]:
# Results Vanilla #
#-----------------#
Train_Vanilla = y_hat_train_Vanilla - y_train
Test_Vanilla = y_hat_test_Vanilla - y_test
score_Train_Vanilla = np.mean(np.abs(Train_Vanilla))
score_Test_Vanilla = np.mean(np.abs(Test_Vanilla))
score_Train_Vanilla_RMSE = np.mean((Train_Vanilla)**2)
score_Test_Vanilla_RMSE = np.mean((Test_Vanilla)**2)

In [13]:
# Performance Metrics
#----------------------#
performance_out = pd.DataFrame({
'Good': np.array([np.mean(score_Train_good),score_Train_good_RMSE,np.mean(score_Test_good),score_Test_good_RMSE]),
'Bad': np.array([np.mean(score_Train_bad),score_Train_bad_RMSE,np.mean(score_Test_bad),score_Test_bad_RMSE]),
'Vanilla': np.array([np.mean(score_Train_Vanilla),score_Train_Vanilla_RMSE,np.mean(score_Test_Vanilla),score_Test_Vanilla_RMSE])
},index=['MAE: Train','MAE: Test','RMSE: Train','RMSE: Test'])

# Write Results
#---------------#
# LaTeX
performance_out.to_latex('./outputs/results/Performance.txt')
# Write to Txt
cur_path = os.path.expanduser('./outputs/results/Performance_text.txt')
with open(cur_path, "w") as f:
    f.write(str(performance_out))

# Live Readings

In [14]:
print('Et-Voila!')
print(performance_out)

Et-Voila!
                  Good         Bad    Vanilla
MAE: Train    0.381672    2.073648   0.428122
MAE: Test     0.316039    5.637205   0.375244
RMSE: Train   0.420023    2.056685   0.434948
RMSE: Test    0.374318    5.548146   0.391001
MAPE: Train  76.622347  100.000000  74.287018
MAPE: Test   76.598350  100.000000  74.304050


---
#### 😊 Fin 😊
---